<a href="https://colab.research.google.com/github/faezesarlakifar/AllerTrans/blob/main/modeling/classic_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @markdown import necessaries
from google.colab import drive
from tqdm.notebook import tqdm
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# @markdown mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Load Dataset
path = '/content/drive/MyDrive/allergen-detection/embeddings/esm-embeddings/'
df_train = pd.read_csv(path+'df_train.csv', index_col=0)
df_test = pd.read_csv(path+'df_test.csv', index_col=0)

In [ ]:
# Convert Label to integer
df_train['Label'] = df_train['Label'].astype(int)
df_test['Label'] = df_test['Label'].astype(int)

# Split data into train and validation sets
X_train = df_train.drop('Label', axis=1)
y_train = df_train['Label']
X_test = df_test.drop('Label', axis=1)
y_test = df_test['Label']

In [ ]:
df_test.head()

,0,1,2,3,4,5,6,7,8,9,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,Label
0,-0.022788,0.102351,-0.004759,0.056018,0.041625,-0.091652,-0.132269,0.186708,0.009317,-0.013024,...,0.058825,-0.126656,0.029039,-0.043456,0.176962,0.107510,0.044039,-0.119478,-0.009796,0
1,-0.028571,-0.017322,-0.037169,0.027144,0.002909,-0.105564,0.061668,-0.166686,-0.007113,0.075881,...,-0.024243,0.019002,0.035870,-0.109937,0.029677,0.092280,-0.143162,-0.092381,0.013006,1
2,0.067068,-0.074117,0.011278,0.056075,-0.099463,-0.078861,0.021225,-0.012659,0.035010,-0.041538,...,-0.014853,0.003386,0.025327,-0.010808,0.033378,0.120301,-0.004345,-0.005355,0.035297,0
3,-0.029574,-0.049990,-0.052827,-0.066867,0.070059,-0.028007,-0.048950,-0.046356,-0.165470,0.093307,...,0.026353,-0.027442,-0.003394,-0.074090,-0.126639,-0.044253,-0.181208,0.106327,0.195119,0
4,0.016483,-0.101985,0.016983,0.041254,-0.012146,-0.078855,-0.021125,-0.142708,0.039837,0.017908,...,-0.008563,-0.002452,-0.037658,-0.044058,0.018399,0.075191,-0.139385,0.036831,0.094892,1


In [ ]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,Label
0,0.007024,-0.065006,-0.028041,0.111512,0.010963,0.050427,0.117898,-0.123601,0.096952,0.053966,...,-0.064849,0.025921,0.108775,-0.035739,-0.080950,0.042518,-0.047726,-0.028700,0.069091,1
1,-0.016422,0.017157,0.065902,-0.045502,-0.020965,0.019144,-0.008605,0.107032,0.024592,0.059676,...,0.036166,-0.074265,0.072319,-0.044529,0.162673,0.107594,0.038635,-0.063711,0.072911,0
2,-0.067091,0.036359,-0.012731,0.002579,0.015604,-0.163380,0.009639,-0.003973,-0.108023,0.069293,...,0.014872,-0.137274,0.086011,-0.035893,0.001509,0.090051,-0.043187,0.003108,0.034345,1
3,0.003069,-0.012822,-0.020514,0.109759,-0.071700,0.016879,0.031743,0.002377,0.093078,-0.009462,...,0.098554,0.046076,0.111619,-0.035462,0.100664,0.147820,0.014518,-0.152727,-0.041294,1
4,0.045561,-0.089433,-0.039230,0.128283,-0.066253,-0.094849,0.142744,-0.094645,0.017515,0.111390,...,0.039378,-0.177669,-0.003415,0.036854,-0.080315,-0.023137,0.031750,0.065749,0.070150,0


# Preprocess Data

In [ ]:
# Create new feature names
feature_names = ['feat_'+str(i) for i in range(X_train.shape[1])]

# Rename columns
X_train.columns = feature_names
X_test.columns = feature_names

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Convert to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Train Classifiers

In [ ]:
# Train classifiers
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

svm = SVC(gamma='auto')
svm.fit(X_train, y_train)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

xgb = XGBClassifier(n_estimators=100, learning_rate=0.1)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
# Evaluate on test set
y_rf_pred = rf.predict(X_test)
y_svm_pred = svm.predict(X_test)
y_knn_pred = knn.predict(X_test)
y_xgb_pred = xgb.predict(X_test)

### Reported results (1)

In [ ]:
print("Random Forest Test Accuracy: ", accuracy_score(y_test, y_rf_pred))
print('Random Forest Test Result:\n', classification_report(y_test , y_rf_pred ))

print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('SVM Test Result:\n', classification_report(y_test , y_svm_pred ))

print("KNN Test Accuracy: ", accuracy_score(y_test, y_knn_pred))
print('KNN Test Result:\n', classification_report(y_test , y_knn_pred ))

print("XGBoost Test Accuracy: ", accuracy_score(y_test, y_xgb_pred))
print('XGBoost Test Result:\n', classification_report(y_test , y_xgb_pred ))

Random Forest Test Accuracy:  0.7722084367245657
Random Forest Test Result:
               precision    recall  f1-score   support

           0       0.69      1.00      0.81      2015
           1       0.99      0.55      0.71      2015

    accuracy                           0.77      4030
   macro avg       0.84      0.77      0.76      4030
weighted avg       0.84      0.77      0.76      4030

SVM Test Accuracy:  0.9307692307692308
SVM Test Result:
               precision    recall  f1-score   support

           0       0.92      0.95      0.93      2015
           1       0.94      0.92      0.93      2015

    accuracy                           0.93      4030
   macro avg       0.93      0.93      0.93      4030
weighted avg       0.93      0.93      0.93      4030

KNN Test Accuracy:  0.8593052109181142
KNN Test Result:
               precision    recall  f1-score   support

           0       0.80      0.97      0.87      2015
           1       0.96      0.75      0.84   

# Hyper-Parameter Tuning

In [ ]:
param_grid = {'kernel': ['rbf', 'linear', 'poly'],
              'C': [0.1, 1, 10],
              'gamma': [0.01, 0.1, 1]}

grid = GridSearchCV(SVC(), param_grid, cv=5)
grid.fit(X_train, y_train)

print("Best parameters:", grid.best_params_)
print("Best accuracy:", grid.best_score_)

Best parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Best accuracy: 0.9879652605459057


In [ ]:
df_comparative_results.to_csv('comparative_results.csv')

In [ ]:
svm = SVC(kernel='poly', gamma=0.01, C=0.5)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.9305210918114144
svm result:
               precision    recall  f1-score   support

           0       0.91      0.96      0.93      2015
           1       0.96      0.90      0.93      2015

    accuracy                           0.93      4030
   macro avg       0.93      0.93      0.93      4030
weighted avg       0.93      0.93      0.93      4030



In [ ]:
svm = SVC(kernel='rbf', gamma=0.01, C=0.5)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.9399503722084367
svm result:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94      2015
           1       0.96      0.92      0.94      2015

    accuracy                           0.94      4030
   macro avg       0.94      0.94      0.94      4030
weighted avg       0.94      0.94      0.94      4030



In [ ]:
svm = SVC(kernel='rbf', gamma=0.1, C=0.5)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.890074441687345
svm result:
               precision    recall  f1-score   support

           0       0.83      0.98      0.90      2015
           1       0.98      0.80      0.88      2015

    accuracy                           0.89      4030
   macro avg       0.90      0.89      0.89      4030
weighted avg       0.90      0.89      0.89      4030



In [ ]:
svm = SVC(kernel='poly', gamma='auto', C=1)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.5789081885856079
svm result:
               precision    recall  f1-score   support

           0       0.76      0.23      0.35      2015
           1       0.55      0.93      0.69      2015

    accuracy                           0.58      4030
   macro avg       0.65      0.58      0.52      4030
weighted avg       0.65      0.58      0.52      4030



In [ ]:
svm = SVC(kernel='rbf', gamma=0.001, C=0.45)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.9277915632754342
svm result:
               precision    recall  f1-score   support

           0       0.91      0.95      0.93      2015
           1       0.94      0.91      0.93      2015

    accuracy                           0.93      4030
   macro avg       0.93      0.93      0.93      4030
weighted avg       0.93      0.93      0.93      4030



In [ ]:
svm = SVC(kernel='rbf', gamma=0.01, C=0.3)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.9382133995037221
svm result:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94      2015
           1       0.96      0.92      0.94      2015

    accuracy                           0.94      4030
   macro avg       0.94      0.94      0.94      4030
weighted avg       0.94      0.94      0.94      4030



In [ ]:
svm = SVC(kernel='rbf', gamma=0.01, C=0.4)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.9397022332506203
svm result:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94      2015
           1       0.96      0.92      0.94      2015

    accuracy                           0.94      4030
   macro avg       0.94      0.94      0.94      4030
weighted avg       0.94      0.94      0.94      4030



In [ ]:
svm = SVC(kernel='rbf', gamma=0.01, C=0.55)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.9406947890818859
svm result:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94      2015
           1       0.96      0.92      0.94      2015

    accuracy                           0.94      4030
   macro avg       0.94      0.94      0.94      4030
weighted avg       0.94      0.94      0.94      4030



In [ ]:
svm = SVC(kernel='rbf', gamma=0.02, C=0.5)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.9377171215880893
svm result:
               precision    recall  f1-score   support

           0       0.91      0.97      0.94      2015
           1       0.97      0.90      0.94      2015

    accuracy                           0.94      4030
   macro avg       0.94      0.94      0.94      4030
weighted avg       0.94      0.94      0.94      4030



In [ ]:
svm = SVC(kernel='rbf', gamma=0.01, C=0.6)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.9404466501240695
svm result:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94      2015
           1       0.96      0.92      0.94      2015

    accuracy                           0.94      4030
   macro avg       0.94      0.94      0.94      4030
weighted avg       0.94      0.94      0.94      4030



In [ ]:
svm = SVC(kernel='rbf', gamma=0.01, C=0.57)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.9406947890818859
svm result:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94      2015
           1       0.96      0.92      0.94      2015

    accuracy                           0.94      4030
   macro avg       0.94      0.94      0.94      4030
weighted avg       0.94      0.94      0.94      4030



In [ ]:
svm = SVC(kernel='rbf', gamma=0.009, C=0.58)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.9401985111662531
svm result:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94      2015
           1       0.96      0.92      0.94      2015

    accuracy                           0.94      4030
   macro avg       0.94      0.94      0.94      4030
weighted avg       0.94      0.94      0.94      4030



In [ ]:
svm = SVC(kernel='rbf', gamma='scale', C=0.58, random_state=42)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.9379652605459057
svm result:
               precision    recall  f1-score   support

           0       0.91      0.97      0.94      2015
           1       0.97      0.91      0.94      2015

    accuracy                           0.94      4030
   macro avg       0.94      0.94      0.94      4030
weighted avg       0.94      0.94      0.94      4030



In [ ]:
# @title Reported results (2)
svm = SVC(kernel='rbf', gamma=0.01, C=0.58, degree=7)
svm.fit(X_train, y_train)

y_svm_pred = svm.predict(X_test)
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.9406947890818859
svm result:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94      2015
           1       0.96      0.92      0.94      2015

    accuracy                           0.94      4030
   macro avg       0.94      0.94      0.94      4030
weighted avg       0.94      0.94      0.94      4030



In [ ]:
params = {
 'learning_rate' : [0.05,0.10,0.15,0.20,0.25,0.30],
 'max_depth' : [ 3, 4, 5, 6, 8, 10, 12, 15],
 'min_child_weight' : [ 1, 3, 5, 7 ],
 'gamma': [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 'colsample_bytree' : [ 0.3, 0.4, 0.5 , 0.7 ]
}

rs_model=RandomizedSearchCV(XGBClassifier(),
                            param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)
rs_model.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5,
                                                             0.7],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2,
                                                          0.25, 0.3],
                                        'max_depth': [3, 4, 5, 6, 8, 10, 12,
                                                      15],
                                        'min_child_weight': [1, 3, 5, 7]},
                   scoring='roc_auc', verbose=3)

In [ ]:
print("Best parameters:", rs_model.best_estimator_)

Best parameters: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.4, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.25, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)


In [ ]:
xgb = rs_model.best_estimator_

In [ ]:
y_xgb_pred = xgb.predict(X_test)

In [ ]:
print("XGBoost Test Accuracy: ", accuracy_score(y_test, y_xgb_pred))
print('XGBoost Test Result:\n', classification_report(y_test , y_xgb_pred ))

XGBoost Test Accuracy:  0.8694789081885856
XGBoost Test Result:
               precision    recall  f1-score   support

           0       0.80      0.99      0.88      2015
           1       0.99      0.75      0.85      2015

    accuracy                           0.87      4030
   macro avg       0.89      0.87      0.87      4030
weighted avg       0.89      0.87      0.87      4030



In [ ]:
xgb = XGBClassifier(n_estimators=100, learning_rate=0.25, gamma=0.5)
xgb.fit(X_train, y_train)
y_xgb_pred = xgb.predict(X_test)
print("XGBoost Test Accuracy: ", accuracy_score(y_test, y_xgb_pred))
print('XGBoost Test Result:\n', classification_report(y_test , y_xgb_pred ))

XGBoost Test Accuracy:  0.8779156327543425
XGBoost Test Result:
               precision    recall  f1-score   support

           0       0.81      0.99      0.89      2015
           1       0.99      0.77      0.86      2015

    accuracy                           0.88      4030
   macro avg       0.90      0.88      0.88      4030
weighted avg       0.90      0.88      0.88      4030



In [ ]:
xgb = XGBClassifier(n_estimators=100, learning_rate=0.01, gamma=0.5)
xgb.fit(X_train, y_train)
y_xgb_pred = xgb.predict(X_test)
print("XGBoost Test Accuracy: ", accuracy_score(y_test, y_xgb_pred))
print('XGBoost Test Result:\n', classification_report(y_test , y_xgb_pred ))

XGBoost Test Accuracy:  0.8516129032258064
XGBoost Test Result:
               precision    recall  f1-score   support

           0       0.78      0.98      0.87      2015
           1       0.97      0.73      0.83      2015

    accuracy                           0.85      4030
   macro avg       0.88      0.85      0.85      4030
weighted avg       0.88      0.85      0.85      4030



# Using AutoEncoder

In [ ]:
# Standardize the data
# scaler = StandardScaler()
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_scaled = torch.from_numpy(X_train_scaled).float()
X_test_scaled = torch.from_numpy(X_test_scaled).float()

In [ ]:
X_train = X_train_scaled
X_test = X_test_scaled

In [ ]:
# Define autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, encoding_dim)
        )

        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
# Create and train autoencoder
model = Autoencoder(X_train_scaled.shape[1], 32)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model.train()
for epoch in range(100):
    recon = model(X_train_scaled)
    loss = criterion(recon, X_train_scaled)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Encode training and test data
X_train_encoded = model.encoder(X_train_scaled)
X_test_encoded = model.encoder(X_test_scaled)

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
# ConvNet Model
class ConvNet(nn.Module):
    def __init__(self, in_channels=1):
      super().__init__()

      # self.conv1 = nn.Conv1d(in_channels, 32, 3)
      # self.conv2 = nn.Conv1d(32, 64, 3)

      self.conv1 = nn.Conv1d(in_channels, 32, 3)
      self.conv2 = nn.Conv1d(32, 64, 3)

      self.conv_dropout = nn.Dropout(p=0.2)

      self.fc1 = nn.Linear(2300 * 32, 128)
      self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
model = ConvNet(1)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters())
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-5)

# Batch size
batch_size = 64

In [ ]:
# Define TensorDataset
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64)


test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64)


In [ ]:
# Training loop
epochs = 15
for epoch in range(epochs):

  for batch in train_loader:

    inputs, labels = batch

    inputs = inputs.unsqueeze(1)
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")



Epoch 1, Loss: 0.1665
Epoch 2, Loss: 0.1430
Epoch 3, Loss: 0.1072
Epoch 4, Loss: 0.0819
Epoch 5, Loss: 0.0920
Epoch 6, Loss: 0.0250
Epoch 7, Loss: 0.0292
Epoch 8, Loss: 0.0049
Epoch 9, Loss: 0.0264
Epoch 10, Loss: 0.0053
Epoch 11, Loss: 0.0317


In [ ]:
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
# Testing loop
results = []
accs = []
y_preds = []
for batch in test_loader:
  inputs, labels = batch
  inputs = inputs.unsqueeze(1)
  # Forward pass
  outputs = model(inputs)

  # Compute accuracy
  _, y_pred = torch.max(outputs, 1)
  # results.append(y_pred == labels)
  accuracy = accuracy_score(labels, y_pred)
  accs.append(accuracy)
  y_preds.extend(y_pred)

# # Overall accuracy
# test_acc = torch.cat(results).mean()

print(sum(accs) / len(accs))
# Metrics
accuracy = accuracy_score(y_test, y_preds)
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_preds, average='weighted')

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", fscore)

In [ ]:
sum(accs) / len(accs)

0.8469022017409114

In [ ]:
# Convert to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Create Tensors from Labels
y_train = torch.from_numpy(y_train).long()
y_test = torch.from_numpy(y_test).long()

In [ ]:
# Train classifiers
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_encoded.detach().numpy(), y_train.detach().numpy())

svm = SVC(gamma='auto')
svm.fit(X_train_encoded.detach().numpy(), y_train.detach().numpy())

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_encoded.detach().numpy(), y_train.detach().numpy())

xgb = XGBClassifier(n_estimators=100, learning_rate=0.1)
xgb.fit(X_train_encoded.detach().numpy(), y_train.detach().numpy())

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

## Reported results (3)

In [ ]:
# Evaluate on test set
y_rf_pred = rf.predict(X_test_encoded.detach().numpy())
y_svm_pred = svm.predict(X_test_encoded.detach().numpy())
y_knn_pred = knn.predict(X_test_encoded.detach().numpy())
y_xgb_pred = xgb.predict(X_test_encoded.detach().numpy())
print("Random Forest Test Accuracy after AutoEncoder: ", accuracy_score(y_test, y_rf_pred))
print("SVM Test Accuracy after AutoEncoder: ", accuracy_score(y_test, y_svm_pred))
print("KNN Test Accuracy after AutoEncoder: ", accuracy_score(y_test, y_knn_pred))
print("XGBoost Test Accuracy after AutoEncoder: ", accuracy_score(y_test, y_xgb_pred))

Random Forest Test Accuracy after AutoEncoder:  0.8831265508684863
SVM Test Accuracy after AutoEncoder:  0.5057071960297767
KNN Test Accuracy after AutoEncoder:  0.8786600496277915
XGBoost Test Accuracy after AutoEncoder:  0.896029776674938


In [ ]:
print('RF result:\n', classification_report(y_test , y_rf_pred ))
print('svm result:\n', classification_report(y_test , y_svm_pred ))
print('knn result:\n', classification_report(y_test , y_knn_pred ))
print('xgb result:\n', classification_report(y_test , y_xgb_pred ))

RF result:
               precision    recall  f1-score   support

           0       0.82      0.98      0.89      2015
           1       0.98      0.78      0.87      2015

    accuracy                           0.88      4030
   macro avg       0.90      0.88      0.88      4030
weighted avg       0.90      0.88      0.88      4030

svm result:
               precision    recall  f1-score   support

           0       0.50      1.00      0.67      2015
           1       1.00      0.01      0.02      2015

    accuracy                           0.51      4030
   macro avg       0.75      0.51      0.35      4030
weighted avg       0.75      0.51      0.35      4030

knn result:
               precision    recall  f1-score   support

           0       0.82      0.97      0.89      2015
           1       0.96      0.79      0.87      2015

    accuracy                           0.88      4030
   macro avg       0.89      0.88      0.88      4030
weighted avg       0.89      0.88   

## More evaluations

In [ ]:
svm = SVC(kernel='rbf', gamma=0.01, C=0.58, degree=7)
svm.fit(X_train_encoded.detach().numpy(), y_train)

y_svm_pred = svm.predict(X_test_encoded.detach().numpy())
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.5245657568238213
svm result:
               precision    recall  f1-score   support

           0       0.51      1.00      0.68      2015
           1       1.00      0.05      0.09      2015

    accuracy                           0.52      4030
   macro avg       0.76      0.52      0.39      4030
weighted avg       0.76      0.52      0.39      4030



In [ ]:
# Create MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=500)

# Train
mlp.fit(X_train_encoded.detach().numpy(), y_train)

# Predict on test set
y_pred = mlp.predict(X_test_encoded.detach().numpy())

# Evaluate
# print("MLP Test accuracy: {:.2f}%".format(mlp.score(X_test, y_test)*100))
print("MLP Test Accuracy: ", accuracy_score(y_test, y_pred))
print('MLP Test Result:\n', classification_report(y_test , y_pred ))

MLP Test Accuracy:  0.9091811414392059
MLP Test Result:
               precision    recall  f1-score   support

           0       0.86      0.98      0.91      2015
           1       0.97      0.84      0.90      2015

    accuracy                           0.91      4030
   macro avg       0.92      0.91      0.91      4030
weighted avg       0.92      0.91      0.91      4030



# Using AutoEncoder v2

In [ ]:
# Create Tensors from Labels
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [ ]:
# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_scaled = torch.from_numpy(X_train_scaled).float()
X_test_scaled = torch.from_numpy(X_test_scaled).float()

In [ ]:
# Define autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, encoding_dim)
        )

        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
# Create and train autoencoder
model = Autoencoder(X_train_scaled.shape[1], 1024)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model.train()
for epoch in range(100):
    recon = model(X_train_scaled)
    loss = criterion(recon, X_train_scaled)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Encode training and test data
X_train_encoded = model.encoder(X_train_scaled)
X_test_encoded = model.encoder(X_test_scaled)

In [ ]:
# Convert to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Create Tensors from Labels
y_train = torch.from_numpy(y_train).long()
y_test = torch.from_numpy(y_test).long()

In [ ]:
# Train classifiers
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_encoded.detach().numpy(), y_train.detach().numpy())

svm = SVC(gamma='auto')
svm.fit(X_train_encoded.detach().numpy(), y_train.detach().numpy())

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_encoded.detach().numpy(), y_train.detach().numpy())

xgb = XGBClassifier(n_estimators=100, learning_rate=0.1)
xgb.fit(X_train_encoded.detach().numpy(), y_train.detach().numpy())

mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=500)
mlp.fit(X_train_encoded.detach().numpy(), y_train)

MLPClassifier(max_iter=500)

In [ ]:
# Evaluate on test set
y_rf_pred = rf.predict(X_test_encoded.detach().numpy())
y_svm_pred = svm.predict(X_test_encoded.detach().numpy())
y_knn_pred = knn.predict(X_test_encoded.detach().numpy())
y_xgb_pred = xgb.predict(X_test_encoded.detach().numpy())
y_mlp_pred = mlp.predict(X_test_encoded.detach().numpy())
print("Random Forest Test Accuracy after AutoEncoder: ", accuracy_score(y_test, y_rf_pred))
print("SVM Test Accuracy after AutoEncoder: ", accuracy_score(y_test, y_svm_pred))
print("KNN Test Accuracy after AutoEncoder: ", accuracy_score(y_test, y_knn_pred))
print("XGBoost Test Accuracy after AutoEncoder: ", accuracy_score(y_test, y_xgb_pred))
print("MLP Test Accuracy after AutoEncoder: ", accuracy_score(y_test, y_mlp_pred))

Random Forest Test Accuracy after AutoEncoder:  0.886848635235732
SVM Test Accuracy after AutoEncoder:  0.5925558312655087
KNN Test Accuracy after AutoEncoder:  0.8774193548387097
XGBoost Test Accuracy after AutoEncoder:  0.9109181141439205
MLP Test Accuracy after AutoEncoder:  0.896029776674938


In [ ]:
print('RF result:\n', classification_report(y_test , y_rf_pred ))
print('svm result:\n', classification_report(y_test , y_svm_pred ))
print('knn result:\n', classification_report(y_test , y_knn_pred ))
print('xgb result:\n', classification_report(y_test , y_xgb_pred ))
print('MLP Result:\n', classification_report(y_test , y_mlp_pred ))

RF result:
               precision    recall  f1-score   support

           0       0.82      0.99      0.90      2015
           1       0.98      0.79      0.87      2015

    accuracy                           0.89      4030
   macro avg       0.90      0.89      0.89      4030
weighted avg       0.90      0.89      0.89      4030

svm result:
               precision    recall  f1-score   support

           0       0.55      1.00      0.71      2015
           1       1.00      0.19      0.31      2015

    accuracy                           0.59      4030
   macro avg       0.77      0.59      0.51      4030
weighted avg       0.77      0.59      0.51      4030

knn result:
               precision    recall  f1-score   support

           0       0.82      0.97      0.89      2015
           1       0.97      0.78      0.86      2015

    accuracy                           0.88      4030
   macro avg       0.89      0.88      0.88      4030
weighted avg       0.89      0.88   

In [ ]:
svm = SVC(kernel='rbf', gamma=0.01, C=0.58, degree=7)
svm.fit(X_train_encoded.detach().numpy(), y_train)

y_svm_pred = svm.predict(X_test_encoded.detach().numpy())
print("SVM Test Accuracy: ", accuracy_score(y_test, y_svm_pred))
print('svm result:\n', classification_report(y_test , y_svm_pred ))

SVM Test Accuracy:  0.5245657568238213
svm result:
               precision    recall  f1-score   support

           0       0.51      1.00      0.68      2015
           1       1.00      0.05      0.09      2015

    accuracy                           0.52      4030
   macro avg       0.76      0.52      0.39      4030
weighted avg       0.76      0.52      0.39      4030



In [ ]:
# Create MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=500)

# Train
mlp.fit(X_train_encoded.detach().numpy(), y_train)

# Predict on test set
y_pred = mlp.predict(X_test_encoded.detach().numpy())

# Evaluate
# print("MLP Test accuracy: {:.2f}%".format(mlp.score(X_test, y_test)*100))
print("MLP Test Accuracy: ", accuracy_score(y_test, y_pred))
print('MLP Test Result:\n', classification_report(y_test , y_pred ))

MLP Test Accuracy:  0.9091811414392059
MLP Test Result:
               precision    recall  f1-score   support

           0       0.86      0.98      0.91      2015
           1       0.97      0.84      0.90      2015

    accuracy                           0.91      4030
   macro avg       0.92      0.91      0.91      4030
weighted avg       0.92      0.91      0.91      4030

